# Tests on the projection onto the set of real-valued matrices with prescribed row and column sums

In [1]:
import numpy  as np
import pandas as pd
import time
import os
import matplotlib.pyplot as plt

# Particular functions
from numpy import zeros, zeros_like, allclose, where, ones, inf, absolute, linspace, tile, maximum, newaxis, broadcast_to
from numpy.random import default_rng as rng
from numba import jit
from scipy.spatial.distance import cdist
from scipy.linalg import norm

Set dimensions of problem and create randomised instance:

In [120]:
N = 50*50
M = 65*65
#N = 90*120
#M = 131*72
n = rng(0).uniform(0.1,1,N)
n/= n.sum()
m = rng(0).uniform(0.1,1,M)
m/= m.sum()
N, M = n.size, m.size
c = (n.reshape(N,1) + m).T

Initialise variables and compute desired result

In [121]:
τ = 1e-3 * 1.9
    
# Initialise σ
σ = 1.0/τ - 1e-5

# Initialise ρ
ρ = 1.9 #- 1e-4 # this helped in 8x8 but not for bigger colour instances

# Fetch lengths of m and n.
N = n.size
M = m.size
    
x, y = zeros((2,M,N)) + 1;
xₖ, yₖ, xₚ, u = zeros((4,M,N))
κ_1, κ_2 = zeros(M), zeros(N)
        
xₖ = x - τ * (c + y);    maximum(xₖ, 0.0, xₖ)

u = y/σ + 2.0*xₖ - x
u += 0.1                # To have at least a non-zero value

κ_1 = u.sum(1) - m
κ_2 = u.sum(0) - n

β_1 = κ_1.sum() / (M + N)
β_2 = κ_2.sum() / (M + N)

yₖ = σ*(tile( (κ_1 - β_1)/N, (N,1)).T + tile( (κ_2 - β_2)/M, (M,1)))

Testing different ways of computing the projection complement:

In [4]:
# Original
print(4, allclose(yₖ, σ*(tile( (κ_1 - β_1)/N, (N,1)).T + tile( (κ_2 - β_2)/M, (M,1))) ))
# Broadcasting between two shapes
print(3, allclose(yₖ, σ*((κ_1 - β_1).reshape(M,1)/N + (κ_2 - β_2)/M) ))
# Broadcasting between two shapes but with pre-multiplication by σ
print(2, allclose(yₖ, ( ( (κ_1 - β_1)*σ/N).reshape(M,1) + σ*(κ_2 - β_2)/M) ))
# Using a new axis object instead of reshaping
print(1, allclose(yₖ, ( ( (κ_1 - β_1)*σ/N)[..., newaxis] + σ*(κ_2 - β_2)/M) ))

4 True
3 True
2 True
1 True


Let's time the different methods:

In [5]:
%timeit -r 10 -n 200  ( ( (κ_1 - β_1)*σ/N)[..., newaxis] + σ*(κ_2 - β_2)/M)

7.32 ms ± 65.6 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


In [6]:
%timeit -r 10 -n 200 ( ( (κ_1 - β_1)*σ/N).reshape(M,1) + σ*(κ_2 - β_2)/M)

7.09 ms ± 12.8 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


In [7]:
%timeit -r 10 -n 200 σ*((κ_1 - β_1).reshape(M,1)/N + (κ_2 - β_2)/M)

10.1 ms ± 13.3 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


In [8]:
%timeit -r 10 -n 200 σ*(tile( (κ_1 - β_1)/N, (N,1)).T + tile( (κ_2 - β_2)/M, (M,1)))

26.8 ms ± 660 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


Alternative by ```broadcast_to```:

In [12]:
print(5, allclose(yₖ, σ*(broadcast_to((κ_1 - β_1)/N, (N, M)).T + broadcast_to((κ_2 - β_2)/M, (M, N))) ) )

5 True


The difference in performance between tiling and broadcasting is impressive:

In [13]:
%timeit -r 10 -n 200 broadcast_to((κ_1 - β_1)/N, (N, M)).T
%timeit -r 10 -n 200 tile( (κ_1 - β_1)/N, (N,1)).T

21.2 µs ± 11.1 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)
3.25 ms ± 98.4 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


However using ```broadcast_to``` does exactly the same as method 2 above (and even has additional overhead).

In [14]:
%timeit -r 10 -n 200 σ*(broadcast_to((κ_1 - β_1)/N, (N, M)).T + broadcast_to((κ_2 - β_2)/M, (M, N)))

10.1 ms ± 57.1 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


---

### Test for reducing complexity

In [99]:
from numpy import add, subtract

In [82]:
def test_1():
    y = ones((M,N));
    yₖ = ( (κ_1 - β_1)*σ/N).reshape(M,1) + σ*(κ_2 - β_2)/M
    y *= 1-ρ;   yₖ *= ρ;     y += yₖ
    
def test_2():
    y = ones((M,N));
    yᵢₖ = (κ_1 - β_1)*ρ*σ/N
    yⱼₖ = (κ_2 - β_2)*ρ*σ/M
    
    y *= 1-ρ
    add(y,yⱼₖ, out=y)
    add(y,yᵢₖ.reshape(M,1), out=y)

$(M,N) = (4225, 2500)$:

In [83]:
%timeit -r 10 -n 10 test_1()
%timeit -r 10 -n 10 test_2()

21 ms ± 950 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
14.7 ms ± 340 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


$(M,N) = (9432, 10800)$:

In [79]:
%timeit -r 10 -n 10 test_1()
%timeit -r 10 -n 10 test_2()

618 ms ± 81.9 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
323 ms ± 14 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [52]:
allclose(np.add(np.add(y,yⱼₖ),yᵢₖ.reshape(M,1)), np.add(y,yₖ))

True

We can further reduce the complexity and memory requirements of using $y$:

In [115]:
def Test_A():
    x, y, xₖ, u = ones((4,M,N));

    xₖ = x - τ * (c + y)
    maximum(xₖ, 0.0, xₖ)

    u = y/σ + 2.0*xₖ - x

    κ_1 = u.sum(1) - m
    κ_2 = u.sum(0) - n

    β_1 = κ_1.sum() / (M + N)
    β_2 = κ_2.sum() / (M + N)

    yₖ = ( (κ_1 - β_1)*σ/N).reshape(M,1) + σ*(κ_2 - β_2)/M                       # Broadcasting

    # Reset x and y for the next iteration
    x *= 1-ρ;   xₖ *= ρ;     x += xₖ
    y *= 1-ρ;   yₖ *= ρ;     y += yₖ

In [116]:
def Test_B():
    x, y, xₖ, u = ones((4,M,N));

    xₖ = x - τ * (c + y)
    maximum(xₖ, 0.0, xₖ)

    u = y/σ + 2.0*xₖ - x

    κ_1 = u.sum(1) - m
    κ_2 = u.sum(0) - n

    β_1 = κ_1.sum() / (M + N)
    β_2 = κ_2.sum() / (M + N)

    yᵢₖ = (ρ*σ/N) * (κ_1 - β_1)
    yⱼₖ = (ρ*σ/M) * (κ_2 - β_2)

    # Reset x and y for the next iteration
    x *= 1-ρ;   xₖ *= ρ;     x += xₖ
    y *= 1-ρ
    add(y,yⱼₖ, out=y)
    add(y,yᵢₖ.reshape(M,1), out=y)

In [117]:
def Test_C():
    x,p,u = ones((3,M,N));
    ϕ = 0.5*ones(M);
    ψ = 0.5*ones(N);
    
    # Primal step
    #p = x - τ * (c + y)
    add(c,ψ, out=p)
    add(p,ϕ.reshape(M,1), out=p)
    maximum(x - τ * p, 0.0, p)
    
    # Dual step
    u = 2.0*p - x
    add(u,ψ/σ, out=u)
    add(u,ϕ.reshape(M,1)/σ, out=u)
    
    # Projection step
    κ_1 = u.sum(1) - m
    κ_2 = u.sum(0) - n

    β_1 = κ_1.sum() / (M + N)
    β_2 = κ_2.sum() / (M + N)

    yᵢ = (ρ*σ/N) * (κ_1 - β_1)
    yⱼ = (ρ*σ/M) * (κ_2 - β_2)

    # Reset x and y for the next iteration
    x *= 1-ρ;   p *= ρ;     x += p
    ϕ *= 1-ρ;   ϕ += yᵢ
    ψ *= 1-ρ;   ψ += yⱼ
    
def Test_D():
    x,p,u = ones((3,M,N));
    ϕ = 0.5*ones(M);
    ψ = 0.5*ones(N);
    
    # Primal step
    #p = x - τ * (c + y)
    add(c,ψ, out=p)
    add(p,ϕ.reshape(M,1), out=p)
    maximum(x - τ * p, 0.0, p)
    
    # Dual step
    subtract(2.0*p, x, out=u)
    add(u,ψ/σ, out=u)
    add(u,ϕ.reshape(M,1)/σ, out=u)
    
    # Projection step
    κ_1 = u.sum(1) - m
    κ_2 = u.sum(0) - n

    β_1 = κ_1.sum() / (M + N)
    β_2 = κ_2.sum() / (M + N)

    yᵢ = (ρ*σ/N) * (κ_1 - β_1)
    yⱼ = (ρ*σ/M) * (κ_2 - β_2)

    # Reset x and y for the next iteration
    x *= 1-ρ;   p *= ρ;     x += p
    ϕ *= 1-ρ;   ϕ += yᵢ
    ψ *= 1-ρ;   ψ += yⱼ
    
def Test_E():
    x,p,u = ones((3,M,N));
    ϕ = 0.5*ones(M);
    ψ = 0.5*ones(N);
    
    # Primal step
    #p = x - τ * (c + y)
    add(c,ψ, out=p)
    add(p,ϕ.reshape(M,1), out=p)
    subtract(x, τ*p, out=p)
    maximum(p, 0.0, p)
    
    # Dual step
    subtract(2.0*p, x, out=u)
    add(u,ψ/σ, out=u)
    add(u,ϕ.reshape(M,1)/σ, out=u)
    
    # Projection step
    κ_1 = u.sum(1) - m
    κ_2 = u.sum(0) - n

    β_1 = κ_1.sum() / (M + N)
    β_2 = κ_2.sum() / (M + N)

    yᵢ = (ρ*σ/N) * (κ_1 - β_1)
    yⱼ = (ρ*σ/M) * (κ_2 - β_2)

    # Reset x and y for the next iteration
    x *= 1-ρ;   p *= ρ;     x += p
    ϕ *= 1-ρ;   ϕ += yᵢ
    ψ *= 1-ρ;   ψ += yⱼ

$(M,N) = (4225, 2500)$:

In [98]:
%timeit -r 10 -n 20 Test_A()
%timeit -r 10 -n 20 Test_B()
%timeit -r 10 -n 20 Test_C()

180 ms ± 6.69 ms per loop (mean ± std. dev. of 10 runs, 20 loops each)
172 ms ± 5.44 ms per loop (mean ± std. dev. of 10 runs, 20 loops each)
145 ms ± 7.77 ms per loop (mean ± std. dev. of 10 runs, 20 loops each)


In [107]:
%timeit -r 10 -n 20 Test_C()
%timeit -r 10 -n 20 Test_D()
%timeit -r 10 -n 20 Test_E()

144 ms ± 6.96 ms per loop (mean ± std. dev. of 10 runs, 20 loops each)
147 ms ± 4.13 ms per loop (mean ± std. dev. of 10 runs, 20 loops each)
145 ms ± 2.05 ms per loop (mean ± std. dev. of 10 runs, 20 loops each)


$(M,N) = (9432, 10800)$:

In [118]:
%timeit -r 10 -n 20 Test_A()
%timeit -r 10 -n 20 Test_B()
%timeit -r 10 -n 20 Test_C()

5.4 s ± 796 ms per loop (mean ± std. dev. of 10 runs, 20 loops each)
4.99 s ± 400 ms per loop (mean ± std. dev. of 10 runs, 20 loops each)
2.78 s ± 117 ms per loop (mean ± std. dev. of 10 runs, 20 loops each)


In [119]:
%timeit -r 10 -n 20 Test_C()
%timeit -r 10 -n 20 Test_D()
%timeit -r 10 -n 20 Test_E()

2.86 s ± 147 ms per loop (mean ± std. dev. of 10 runs, 20 loops each)
2.86 s ± 63.4 ms per loop (mean ± std. dev. of 10 runs, 20 loops each)
2.54 s ± 120 ms per loop (mean ± std. dev. of 10 runs, 20 loops each)


$(M,N) = (64, 64)$:

In [125]:
%timeit -r 10 -n 20 Test_A()
%timeit -r 10 -n 20 Test_B()
%timeit -r 10 -n 20 Test_C()

153 µs ± 69.6 µs per loop (mean ± std. dev. of 10 runs, 20 loops each)
98.9 µs ± 10.6 µs per loop (mean ± std. dev. of 10 runs, 20 loops each)
92 µs ± 10.7 µs per loop (mean ± std. dev. of 10 runs, 20 loops each)


In [126]:
%timeit -r 10 -n 20 Test_C()
%timeit -r 10 -n 20 Test_D()
%timeit -r 10 -n 20 Test_E()

164 µs ± 81.7 µs per loop (mean ± std. dev. of 10 runs, 20 loops each)
95.4 µs ± 13 µs per loop (mean ± std. dev. of 10 runs, 20 loops each)
85.4 µs ± 14.4 µs per loop (mean ± std. dev. of 10 runs, 20 loops each)


---